In [8]:
import os
import pandas as pd
import numpy as np

import matplotlib

# import reportlab
# from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
# from reportlab.pdfgen import canvas
# from reportlab.lib.pagesizes import letter
# from reportlab.lib import utils
# from reportlab.lib import colors
# from reportlab.pdfbase import pdfmetrics
# from reportlab.pdfbase.ttfonts import TTFont

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors, utils
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

In [2]:
#Get current and parent directory directory
current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)

#Path to the source folder where the raw data is located
source_path = os.path.join(parent_directory, 'data', 'raw', 'data_per_day.xlsx')

#Path to the image
image_path = os.path.join(parent_directory, 'graphics', 'bow.png')

#Path to the destination file
destination_path = os.path.join(parent_directory, 'data', 'processed', 'output.pdf')

#Read the .xlsx file
df = pd.read_excel(source_path, index_col=None)

In [3]:
#Main distinction is saved in the system as 'NA' and 'NNA'. 'NA' is read as a empty value.
#We need fill 'empty' value as a teachers, and change 'NNA' as a not teachers.
df['Grupa pracowników'] = df['Grupa pracowników'].fillna('Nauczyciel')
df['Grupa pracowników'] = df['Grupa pracowników'].replace('NNA', 'Nienauczyciel')

display(df)

,Numer osobowy,Pion,Obszar kadrowy,Podobszar kadrowy,Podgrupa pracowników,Stanowisko,Etat,Płeć,Tytuł,Grupa pracowników,Wiek,Wynagrodzenie
0,Person 1310,Rektor,Wydział 1,Wydział 1,Dydaktyczni,adiunkt,1.0,Mężczyzna,doktor,Nauczyciel,41,7839
1,Person 1312,Rektor,Wydział 1,Wydział 1,Badawczo-dydaktyczni,adiunkt,1.0,Mężczyzna,doktor,Nauczyciel,38,7415
2,Person 1314,Rektor,Wydział 1,Wydział 1,Badawczo-dydaktyczni,adiunkt,1.0,Mężczyzna,dr habilitowany,Nauczyciel,56,6389
3,Person 1315,Rektor,Wydział 1,Wydział 1,Badawczo-dydaktyczni,adiunkt,1.0,Mężczyzna,doktor,Nauczyciel,55,6987
4,Person 1318,Rektor,Wydział 1,Wydział 1,Badawczo-dydaktyczni,adiunkt,1.0,Kobieta,doktor,Nauczyciel,36,6064
...,...,...,...,...,...,...,...,...,...,...,...,...
1402,Person 1031,Rektor,Wydział 4,Wydział 4 - instytut 2,Dydaktyczni,starszy asystent,1.0,Kobieta,mgr inżynier,Nauczyciel,61,6450
1403,Person 1042,Rektor,Wydział 4,Wydział 4 - instytut 2,Dydaktyczni,starszy asystent,1.0,Mężczyzna,mgr inżynier,Nauczyciel,38,5029
1404,Person 1058,Rektor,Wydział 4,Wydział 4 - instytut 2,Dydaktyczni,starszy asystent,1.0,Mężczyzna,mgr inżynier,Nauczyciel,40,6057
1405,Person 1159,Rektor,Wydział 5,Wydział 5 - instytut 1,Dydaktyczni,starszy asystent,1.0,Mężczyzna,mgr inżynier,Nauczyciel,50,5369


In [4]:
selected_columns = ['Pion'] #, 'Obszar kadrowy']

In [5]:
for col in selected_columns:
    for val in df[col].unique():
        df_pivot_mainGroups = df[df[col] == val].pivot_table(values='Etat', index='Grupa pracowników', columns=col, 
                              aggfunc=['count', 'sum']).applymap(lambda x: round(x, 2)) #.style.background_gradient(cmap='Greens')
        df_pivot_mainGroups.columns = df_pivot_mainGroups.columns.droplevel(1)
        df_pivot_mainGroups.columns.name = None
        df_pivot_mainGroups = df_pivot_mainGroups.rename(columns={'count': 'Osoby', 'sum': 'Etaty'})
        
        
        
        df_pivot_teachersJobPosition = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                       'Etat', index='Stanowisko', columns=col, aggfunc=['count', 'sum']) \
                                       .applymap(lambda x: round(x, 2))
        df_pivot_teachersJobPosition.columns = df_pivot_teachersJobPosition.columns.droplevel(1)
        df_pivot_teachersJobPosition.columns.name = None
        df_pivot_teachersJobPosition = df_pivot_teachersJobPosition.rename(columns={'count': 'Osoby', 'sum': 'Etaty'})
        
        
        df_pivot_teachersSubgroup = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                    'Etat', index='Podgrupa pracowników', columns=col, aggfunc=['count', 'sum']) \
                                    .applymap(lambda x: round(x, 2))
        df_pivot_teachersSubgroup.columns = df_pivot_teachersSubgroup.columns.droplevel(1)
        df_pivot_teachersSubgroup.columns.name = None
        df_pivot_teachersSubgroup = df_pivot_teachersSubgroup.rename(columns={'count': 'Osoby', 'sum': 'Etaty'})
                               
            
        
        df_pivot_notteachersSubgroup = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nienauczyciel')].pivot_table(values=
                                       'Etat', index='Podgrupa pracowników', columns=col, aggfunc=['count', 'sum']) \
                                       .applymap(lambda x: round(x, 2))
        df_pivot_notteachersSubgroup.columns = df_pivot_notteachersSubgroup.columns.droplevel(1)
        df_pivot_notteachersSubgroup.columns.name = None
        df_pivot_notteachersSubgroup = df_pivot_notteachersSubgroup.rename(columns={'count': 'Osoby', 'sum': 'Etaty'})
        
        
        df_teachersSalary = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                       'Wynagrodzenie', index=['Podgrupa pracowników', 'Stanowisko'], columns=col, aggfunc=['count', 'mean', 'median']) \
                                       .applymap(lambda x: round(x, 2)) \
        
        df_teachersSalary.columns = df_teachersSalary.columns.droplevel(1)
        df_teachersSalary.columns.name = None
        df_teachersSalary = df_teachersSalary.rename(columns={'count': 'Osoby', 'mean': 'Średnia', 'median': 'Mediana'})
        df_teachersSalary[['Średnia', 'Mediana']] = df_teachersSalary[['Średnia', 'Mediana']].apply(lambda col: col.apply(lambda x: '{:,.2f} zł'.format(x)))
        
        
        df_notteachersSalary = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nienauczyciel')].pivot_table(values=
                                       'Wynagrodzenie', index=['Podgrupa pracowników'], columns=col, aggfunc=['count', 'mean', 'median']) \
                                       .applymap(lambda x: round(x, 2)) \
        
        df_notteachersSalary.columns = df_notteachersSalary.columns.droplevel(1)
        df_notteachersSalary.columns.name = None
        df_notteachersSalary = df_notteachersSalary.rename(columns={'count': 'Osoby', 'mean': 'Średnia', 'median': 'Mediana'})
        df_notteachersSalary[['Średnia', 'Mediana']] = df_notteachersSalary[['Średnia', 'Mediana']].apply(lambda col: col.apply(lambda x: '{:,.2f} zł'.format(x)))
        
        
        
        display(col, val, df_pivot_mainGroups, df_pivot_teachersJobPosition, df_pivot_teachersSubgroup, df_pivot_notteachersSubgroup, df_teachersSalary, df_notteachersSalary)

'Pion'

'Rektor'

,Osoby,Etaty
Grupa pracowników,,
Nauczyciel,942,839.48
Nienauczyciel,465,416.70


,Osoby,Etaty
Stanowisko,,
adiunkt,499,458.11
asystent,174,131.89
docent,3,3.00
profesor,103,91.98
profesor uczelni,128,124.90
starszy asystent,35,29.60


,Osoby,Etaty
Podgrupa pracowników,,
Badawczo-dydaktyczni,694,634.13
Badawczy,67,61.80
Dydaktyczni,181,143.55


,Osoby,Etaty
Podgrupa pracowników,,
Administracyjno-ekon,208,200.60
Bibliotekarze,7,7.00
Informatycy,12,10.35
Inżynieryjno-techn.,117,85.70
Naukowo-badawczy,10,6.95
Naukowo-techniczni,13,8.60
Obsługa,85,84.50
Robotnicy,13,13.00


Osoby       Średnia       Mediana
Podgrupa pracowników Stanowisko                                         
Badawczo-dydaktyczni adiunkt             367   7,043.83 zł   7,093.00 zł
                     asystent            117   4,941.76 zł   4,921.00 zł
                     profesor             95  12,444.25 zł  12,428.00 zł
                     profesor uczelni    115  10,043.18 zł  10,078.00 zł
Badawczy             adiunkt              37   6,839.14 zł   6,784.00 zł
                     asystent             25   4,930.20 zł   4,775.00 zł
                     profesor              3  13,020.33 zł  14,246.00 zł
                     profesor uczelni      2   8,697.00 zł   8,697.00 zł
Dydaktyczni          adiunkt              95   6,933.75 zł   6,962.00 zł
                     asystent             32   5,209.66 zł   5,291.50 zł
                     docent                3   6,660.33 zł   6,683.00 zł
                     profesor              5  12,449.40 zł  12,901.00 zł
                     profesor uczelni     11   9,980.36 zł  10,093.00 zł
                     starszy asystent     35   5,893.77 zł   5,959.00 zł

,Osoby,Średnia,Mediana
Podgrupa pracowników,,,
Administracyjno-ekon,208,"6,331.96 zł","6,307.50 zł"
Bibliotekarze,7,"5,771.57 zł","5,553.00 zł"
Informatycy,12,"5,931.75 zł","6,061.00 zł"
Inżynieryjno-techn.,117,"6,063.58 zł","6,068.00 zł"
Naukowo-badawczy,10,"6,070.00 zł","6,217.50 zł"
Naukowo-techniczni,13,"6,188.08 zł","5,952.00 zł"
Obsługa,85,"4,720.21 zł","4,653.00 zł"
Robotnicy,13,"4,726.08 zł","4,733.00 zł"


In [6]:
# Tworzenie pliku PDF
c = canvas.Canvas(destination_path, pagesize=letter)
c.setFont('Helvetica-Bold', 20)  # Ustawienie używanego fontu

# Dodanie tekstu
c.drawString(235, 735, "HR - employee stats")

# Dodanie obrazu
img = utils.ImageReader(image_path)
c.drawImage(img, 205, 730, 25, 25)

#Dodanie jednego z pivot
table = reportlab.platypus.Table(df_pivot_mainGroups.to_html().split("\n"))
#pdf_doc = SimpleDocTemplate(output_filename, pagesize=letter)
pdf_table = Table([df_pivot_mainGroups.columns] + df_pivot_mainGroups.values.tolist(), repeatRows=1)
c.draw(pdf_table)

# Zapisanie pliku PDF
c.save()

NameError: name 'utils' is not defined

In [12]:
# Tworzenie pliku PDF
c = canvas.Canvas(destination_path, pagesize=letter)
c.setFont('Helvetica-Bold', 20)  # Ustawienie używanego fontu

# Dodanie tekstu
c.drawString(235, 735, "HR - employee stats")

# Dodanie obrazu
img = utils.ImageReader(image_path)
c.drawImage(img, 205, 730, 25, 25)

# Utwórz obiekt Canvas na stronie o formacie letter (8.5 x 11 cali)
pdf_doc = SimpleDocTemplate(destination_path, pagesize=letter)
pdf_table = Table([df_pivot_mainGroups.columns] + df_pivot_mainGroups.values.tolist(), repeatRows=1)

# Zdefiniuj styl tabeli
style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                    ('GRID', (0, 0), (-1, -1), 1, colors.black)])

pdf_table.setStyle(style)

# Dodaj tabelę do pliku PDF
pdf_doc.build([pdf_table])

# Zapisanie pliku PDF
c.save()